In [ ]:
!pip install huggingface_hub
!pip install ipywidgets

In [3]:
!{sys.executable} -m pip install --upgrade transformers accelerate

/bin/bash: line 1: {sys.executable}: command not found


In [9]:
import sys
!{sys.executable} -m pip install accelerate

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from huggingface_hub import login
login(token="")

In [5]:
import pandas as pd
import re

dev= pd.read_json("../data/wsd/bin_dev.json")
dev.head()

,context_id,context,word,sense,lemma_id,gloss_id,label
0,context.321,والجراثيم المسُبّبة للأمراض مختلفة باختلاف المرض،,للأمراض,كلّ ما خرج بالكائن الحيّ عن حدِّ الصِّحَّة وال...,ar.320,gloss.340,1
1,context.321,والجراثيم المسُبّبة للأمراض مختلفة باختلاف المرض،,للأمراض,مَرَض يمكن أن ينتقل من كائن حيّ إلى كائن آخر ب...,ar.320,gloss.341,0
2,context.321,والجراثيم المسُبّبة للأمراض مختلفة باختلاف المرض،,للأمراض,أمراض مُعديَّة تنتقل عن طريق الاتِّصال الجنسيّ...,ar.320,gloss.342,0
3,context.321,والجراثيم المسُبّبة للأمراض مختلفة باختلاف المرض،,للأمراض,فرع من الطبِّ يُعنى بدراسة أنواع الأمراض، وأسب...,ar.320,gloss.343,0
4,context.321,والجراثيم المسُبّبة للأمراض مختلفة باختلاف المرض،,للأمراض,شذوذات في نموِّ النَّبات وتطوُّره تسبِّبها عضو...,ar.320,gloss.344,0


In [6]:
def preprocess_text(data):
  for i in range(len(data)):
    data.loc[i,'context']=data.loc[i,'context']
    data.loc[i,'sense']=data.loc[i,'sense']
    word=data.loc[i,'word']
    data.loc[i,'context'] = data.loc[i,'context'].replace(word, f"<token>{word}</token>")
    # break
  return data

dev=preprocess_text(dev)

In [7]:
dev.head()

,context_id,context,word,sense,lemma_id,gloss_id,label
0,context.321,والجراثيم المسُبّبة <token>للأمراض</token> مخت...,للأمراض,كلّ ما خرج بالكائن الحيّ عن حدِّ الصِّحَّة وال...,ar.320,gloss.340,1
1,context.321,والجراثيم المسُبّبة <token>للأمراض</token> مخت...,للأمراض,مَرَض يمكن أن ينتقل من كائن حيّ إلى كائن آخر ب...,ar.320,gloss.341,0
2,context.321,والجراثيم المسُبّبة <token>للأمراض</token> مخت...,للأمراض,أمراض مُعديَّة تنتقل عن طريق الاتِّصال الجنسيّ...,ar.320,gloss.342,0
3,context.321,والجراثيم المسُبّبة <token>للأمراض</token> مخت...,للأمراض,فرع من الطبِّ يُعنى بدراسة أنواع الأمراض، وأسب...,ar.320,gloss.343,0
4,context.321,والجراثيم المسُبّبة <token>للأمراض</token> مخت...,للأمراض,شذوذات في نموِّ النَّبات وتطوُّره تسبِّبها عضو...,ar.320,gloss.344,0


In [8]:
len(dev)

14834

In [9]:
from tqdm import tqdm
import time

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"  # Use only GPUs 0, 1, 2

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-27b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-27b",
    device_map="auto",
    torch_dtype="auto",  # Use mixed precision if available
    #device_map="balanced",  # <--- splits across all available GPUs
)


Loading checkpoint shards:   0%|          | 0/24 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [10]:

results = []
for i in tqdm(range(len(dev))):

    prompt = f"""
        You are a model specialized in Arabic word sense disambiguation.
        Given a context, a target word, and a candidate sense (all provided in Arabic),
        determine if the candidate sense correctly matches the meaning of the target word in the given context.

        The target word appears between <token> and </token> tags in the context.

        You MUST respond with exactly one character:
        - Output "1" if the candidate sense matches the meaning of the word in the context.
        - Otherwise, output "0".
        Do NOT output anything else.
        Do NOT output \n or whitespaces.

        Word: {dev['word'][i]}
        Context: {dev['context'][i]}
        Sense: {dev['sense'][i]}
        """

    #start_time = time.time()  # Start timer
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=32)
    results.append(tokenizer.decode(outputs[0]))
    #end_time = time.time()
    #elapsed = end_time - start_time
    
    #print(f"Elapsed time: {elapsed:.4f} seconds")  # Output: ~1.5005 seconds
    #print(tokenizer.decode(outputs[0]))
    
    break

dev['Dev-gemma2-27b'] = results*len(dev)
dev.to_csv('Dev-gemma2:27b.csv', index=False)

  0%|                                              | 0/14834 [02:28<?, ?it/s]


In [1]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-27b",
    device="cuda",  # replace with "mps" to run on a Mac device
)

text = "Once upon a time,"
outputs = pipe(text, max_new_tokens=256)
response = outputs[0]["generated_text"]
print(response)


/home/yyounes/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/24 [00:00<?, ?it/s]

Device set to use cuda


RuntimeError: handle_0 INTERNAL ASSERT FAILED at "../c10/cuda/driver_api.cpp":15, please report a bug to PyTorch. 

In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"  # Use only GPUs 0, 1, 2

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-27b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-27b",
    device_map="auto",
    torch_dtype="auto",  # Use mixed precision if available
    #device_map="balanced",  # <--- splits across all available GPUs
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))
